In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import logging
import datetime

In [2]:
# 로그 생성
logger = logging.getLogger()

# 로그의 출력 기준 설정
logger.setLevel(logging.INFO)

# log 출력 형식
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# log 출력
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

# log를 파일에 출력
file_handler = logging.FileHandler('D:\\통계청 공모전\\코드\\모델학습및예측(lstm)_{:%Y%m%d}.log'.format(datetime.datetime.now()), 
                                   encoding='utf-8')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [3]:
df = pd.read_csv('../최종 전처리.csv')
df

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,digit_1,digit_2,digit_3,Full Text,clean_text
0,S,95.0,952.0,카센터에서 자동차부분정비 타이어오일교환,카 센터 자동차 부분 정비 타이어 오일 교환
1,G,47.0,472.0,상점내에서 일반인을 대상으로 채소 과일판매,상점 내 일반인 대상 채소 과일 판매
2,G,46.0,467.0,절단하여사업체에도매 공업용고무를가지고 합성고무도매,절단 하여사 업체 에도 매 공업 용 고무 가지 고 합성 고무 도매
3,G,47.0,475.0,영업점에서 일반소비자에게 열쇠잠금장치,영업 점 일 반 소비자 열쇠 잠금장치
4,Q,87.0,872.0,어린이집 보호자의 위탁을 받아 취학전아동보육,어린이집 보호자 위탁 받아 취학 전 아동 보육
...,...,...,...,...,...
1099995,NaN,NaN,NaN,사업장에서 일반인대상으로 버섯농장,사업 장 일반인 대상 버섯 농장
1099996,NaN,NaN,NaN,한의원에서 외래환자위주고 치료,한의원 외래 환자 위주 고 치료
1099997,NaN,NaN,NaN,일반점포에서 소비자에게 그림판매,일 반 점포 소비자 그림 판매
1099998,NaN,NaN,NaN,사업장에서 일반인 학생대상으로 학습공간제공,사업 장 일반인 학생 대상 학습 공간 제공


In [4]:
train = df[:1000000]
train

,digit_1,digit_2,digit_3,Full Text,clean_text
0,S,95.0,952.0,카센터에서 자동차부분정비 타이어오일교환,카 센터 자동차 부분 정비 타이어 오일 교환
1,G,47.0,472.0,상점내에서 일반인을 대상으로 채소 과일판매,상점 내 일반인 대상 채소 과일 판매
2,G,46.0,467.0,절단하여사업체에도매 공업용고무를가지고 합성고무도매,절단 하여사 업체 에도 매 공업 용 고무 가지 고 합성 고무 도매
3,G,47.0,475.0,영업점에서 일반소비자에게 열쇠잠금장치,영업 점 일 반 소비자 열쇠 잠금장치
4,Q,87.0,872.0,어린이집 보호자의 위탁을 받아 취학전아동보육,어린이집 보호자 위탁 받아 취학 전 아동 보육
...,...,...,...,...,...
999995,C,13.0,134.0,제품입고 워싱 청바지워싱,제품 입고 워싱 청바지 워싱
999996,F,42.0,424.0,현장에서 고객의요청에의해 실내인테리어,현장 고객 요청 에의 해 실내 인테리어
999997,G,47.0,474.0,영업점에서 일반소비자에게 여성의류 판매,영업 점 일 반 소비자 여성 의류 판매
999998,P,85.0,856.0,사업장에서 일반고객을대상으로 필라테스,사업 장 일 반 고객 대상 필라테스


In [5]:
encoder1 = LabelEncoder()
train['digit_1'] = encoder1.fit_transform(train['digit_1'])
train['digit_1'].nunique()

C:\Users\user\AppData\Local\Temp/ipykernel_10088/3867300257.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['digit_1'] = encoder1.fit_transform(train['digit_1'])


19

In [6]:
encoder2 = LabelEncoder()
train['digit_2'] = encoder2.fit_transform(train['digit_2'])
train['digit_2'].nunique()

C:\Users\user\AppData\Local\Temp/ipykernel_10088/2282110248.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['digit_2'] = encoder2.fit_transform(train['digit_2'])


74

In [7]:
encoder3 = LabelEncoder()
train['digit_3'] = encoder3.fit_transform(train['digit_3'])
train['digit_3'].nunique()

C:\Users\user\AppData\Local\Temp/ipykernel_10088/781095071.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['digit_3'] = encoder3.fit_transform(train['digit_3'])


225

In [8]:
text = np.load('D:/통계청 공모전/text.npy', allow_pickle=True)

# 시퀀스 패딩

In [9]:
logging.info('시퀀스 패딩')

2022-04-13 00:15:48,588 - root - INFO - 시퀀스 패딩


In [10]:
max_len = 21 # 단어 개수 최댓값

text = tf.keras.preprocessing.sequence.pad_sequences(text, maxlen=max_len)
text

array([[   0,    0,    0, ...,  633, 1945, 3832],
       [   0,    0,    0, ...,  296,  147,   12],
       [   0,    0,    0, ..., 1304,  608,   23],
       ...,
       [   0,    0,    0, ..., 1444, 1030,   12],
       [   0,    0,    0, ..., 1434, 1726,   49],
       [   0,    0,    0, ...,   44, 1310,  203]])

In [11]:
train_data = text[:1000000]
test_data = text[1000000:]
target1 = train['digit_1']

X_train, X_test, y_train, y_test = train_test_split(train_data, target1, test_size=0.2, shuffle=True, stratify=target1)

In [12]:
X_train

array([[    0,     0,     0, ...,   962,   359,   213],
       [    0,     0,     0, ...,   876,   912,   368],
       [    0,     0,     0, ...,   161, 10055,  1172],
       ...,
       [    0,     0,     0, ...,  2897,  1676,   208],
       [    0,     0,     0, ...,  1290,     6, 10186],
       [    0,     0,     0, ...,   342,   378,   130]])

# LSTM 학습 (digit_1)

In [13]:
logging.info('digit_1 학습')

2022-04-13 00:15:51,700 - root - INFO - digit_1 학습


In [14]:
vocab_size = 10760
embedding_dim = 128
hidden_units = 128
num_classes = 19 # digit_1 19개 카테고리

model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim))
model1.add(LSTM(hidden_units, return_sequences=True))
model1.add(LSTM(hidden_units))
model1.add(Dropout(0.2))
model1.add(Dense(hidden_units, activation='tanh'))
model1.add(Dense(num_classes, activation='softmax'))

In [15]:
es = EarlyStopping(monitor='accuracy', mode = 'auto', verbose=3, patience=4)
mc1 = ModelCheckpoint('best_model1.h5', monitor='val_accuracy', mode = 'auto', verbose=1, save_best_only=True)

In [16]:
model1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history1 = model1.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc1], validation_split=0.2)

Epoch 1/10
20000/20000 [==============================] - 229s 11ms/step - loss: 0.1526 - accuracy: 0.9582 - val_loss: 0.1055 - val_accuracy: 0.9710

Epoch 00001: val_accuracy improved from -inf to 0.97099, saving model to best_model1.h5
Epoch 2/10
20000/20000 [==============================] - 228s 11ms/step - loss: 0.0938 - accuracy: 0.9739 - val_loss: 0.0952 - val_accuracy: 0.9737

Epoch 00002: val_accuracy improved from 0.97099 to 0.97373, saving model to best_model1.h5
Epoch 3/10
20000/20000 [==============================] - 227s 11ms/step - loss: 0.0792 - accuracy: 0.9777 - val_loss: 0.0950 - val_accuracy: 0.9744

Epoch 00003: val_accuracy improved from 0.97373 to 0.97443, saving model to best_model1.h5
Epoch 4/10
20000/20000 [==============================] - 226s 11ms/step - loss: 0.0684 - accuracy: 0.9806 - val_loss: 0.0961 - val_accuracy: 0.9742

Epoch 00004: val_accuracy did not improve from 0.97443
Epoch 5/10
20000/20000 [==============================] - 226s 11ms/step - 

In [17]:
print("테스트 정확도: %.4f" % (model1.evaluate(X_test, y_test)[1]))

6250/6250 [==============================] - 25s 4ms/step - loss: 0.1195 - accuracy: 0.9730
테스트 정확도: 0.9730


In [18]:
pred1 = model1.predict(X_test)
pred1

array([[6.9883629e-07, 2.4844311e-07, 6.6598644e-04, ..., 6.1408698e-04,
        8.7202556e-05, 8.1487827e-04],
       [3.2612425e-07, 1.1765203e-08, 2.4451787e-04, ..., 1.8926039e-06,
        3.4461982e-07, 1.7408744e-05],
       [7.2490758e-07, 5.3425038e-07, 3.4294712e-06, ..., 1.6159751e-05,
        6.2710856e-04, 9.0763432e-01],
       ...,
       [1.3670962e-06, 6.4736383e-07, 2.4603941e-03, ..., 1.1289111e-05,
        2.4027265e-06, 8.7655553e-06],
       [1.2555717e-04, 1.1325633e-06, 7.0165278e-04, ..., 9.0080666e-06,
        6.5614469e-05, 1.5543710e-04],
       [4.9201390e-07, 1.2400878e-07, 9.9979836e-01, ..., 2.1282709e-07,
        2.8064923e-07, 3.4877053e-06]], dtype=float32)

In [19]:
predict1 = []
for i in range(pred1.shape[0]):
    predict1.append(pred1[i].argmax())
predict1 = np.array(predict1)
predict1

array([ 7,  6, 18, ...,  8,  6,  2], dtype=int64)

In [20]:
print ('Here is the classification report:')
print (classification_report(y_test, predict1))

Here is the classification report:
              precision    recall  f1-score   support

           0       0.67      0.76      0.71       213
           1       0.85      0.78      0.81        85
           2       0.96      0.95      0.96     21038
           3       0.88      0.89      0.89       151
           4       0.85      0.85      0.85       451
           5       0.96      0.94      0.95      7010
           6       0.98      0.98      0.98     49294
           7       0.99      0.99      0.99     19608
           8       0.99      0.99      0.99     37485
           9       0.91      0.93      0.92      2172
          10       0.98      0.98      0.98      2076
          11       0.98      0.99      0.99      8028
          12       0.94      0.92      0.93      5687
          13       0.87      0.89      0.88      3540
          14       0.92      0.92      0.92       593
          15       0.96      0.98      0.97      9322
          16       0.97      0.98      0.98   

# 모델개발용 데이터 예측 (digit_1)

In [21]:
logging.info('모델개발용 자료 digit_1 예측')

2022-04-13 00:54:43,330 - root - INFO - 모델개발용 자료 digit_1 예측


In [22]:
digit1_pred = model1.predict(test_data)
digit1_pred

array([[1.37430929e-08, 7.90150584e-11, 1.02092381e-05, ...,
        1.75690133e-07, 3.80742762e-07, 7.83322633e-08],
       [7.87294880e-07, 4.32514078e-08, 3.07427021e-04, ...,
        1.60957461e-06, 1.15444934e-07, 4.17101137e-06],
       [1.80622350e-09, 1.16270542e-07, 6.31914509e-06, ...,
        1.98250800e-05, 6.60189426e-06, 9.99749243e-01],
       ...,
       [7.20637502e-07, 1.10725594e-07, 2.44410330e-04, ...,
        1.99272176e-06, 2.20471543e-06, 1.05357476e-05],
       [1.96741908e-04, 1.35982418e-05, 1.55816838e-06, ...,
        1.52214825e-05, 9.99191821e-01, 9.67380911e-05],
       [6.50538198e-07, 3.65955835e-08, 2.62947628e-06, ...,
        1.33284868e-06, 7.75577646e-05, 8.09871301e-04]], dtype=float32)

In [23]:
digit1_predict = []
for i in range(digit1_pred.shape[0]):
    digit1_predict.append(digit1_pred[i].argmax())
digit1_predict = np.array(digit1_predict)
digit1_predict

array([ 8,  6, 18, ...,  6, 17, 11], dtype=int64)

In [24]:
digit1_predict = encoder1.inverse_transform(digit1_predict)
digit1_predict

array(['I', 'G', 'S', ..., 'G', 'R', 'L'], dtype=object)

# LSTM 학습 (digit_2)

In [25]:
logging.info('digit_2 학습')

2022-04-13 00:54:53,275 - root - INFO - digit_2 학습


In [26]:
target2 = train['digit_2']
# digit_2에 1개만 가지고 있는 카테고리 존재하므로 stratify=target2하면 에러 발생
X_train, X_test, y_train, y_test = train_test_split(train_data, target2, test_size=0.2, shuffle=True)

In [27]:
vocab_size = 10760
embedding_dim = 128
hidden_units = 128
num_classes = 74 # digit_2 74개 카테고리

model2 = Sequential()
model2.add(Embedding(vocab_size, embedding_dim))
model2.add(LSTM(hidden_units, return_sequences=True))
model2.add(LSTM(hidden_units))
model2.add(Dropout(0.2))
model2.add(Dense(hidden_units, activation='tanh'))
model2.add(Dense(num_classes, activation='softmax'))

In [28]:
# earlystopping은 위에 정의되었으므로 checkpoint만 정의
mc2 = ModelCheckpoint('best_model2.h5', monitor='val_accuracy', mode = 'auto', verbose=1, save_best_only=True)

In [29]:
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history2 = model2.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc2], validation_split=0.2)

Epoch 1/10
20000/20000 [==============================] - 249s 12ms/step - loss: 0.3689 - accuracy: 0.9025 - val_loss: 0.2457 - val_accuracy: 0.9322

Epoch 00001: val_accuracy improved from -inf to 0.93222, saving model to best_model2.h5
Epoch 2/10
20000/20000 [==============================] - 246s 12ms/step - loss: 0.2279 - accuracy: 0.9370 - val_loss: 0.2260 - val_accuracy: 0.9368

Epoch 00002: val_accuracy improved from 0.93222 to 0.93683, saving model to best_model2.h5
Epoch 3/10
20000/20000 [==============================] - 243s 12ms/step - loss: 0.1983 - accuracy: 0.9440 - val_loss: 0.2186 - val_accuracy: 0.9400

Epoch 00003: val_accuracy improved from 0.93683 to 0.93998, saving model to best_model2.h5
Epoch 4/10
20000/20000 [==============================] - 244s 12ms/step - loss: 0.1786 - accuracy: 0.9486 - val_loss: 0.2183 - val_accuracy: 0.9408

Epoch 00004: val_accuracy improved from 0.93998 to 0.94077, saving model to best_model2.h5
Epoch 5/10
20000/20000 [===============

In [30]:
print("테스트 정확도: %.4f" % (model2.evaluate(X_test, y_test)[1]))

6250/6250 [==============================] - 25s 4ms/step - loss: 0.2551 - accuracy: 0.9393
테스트 정확도: 0.9393


In [31]:
pred2 = model2.predict(X_test)
pred2

array([[1.19474022e-08, 4.93510177e-10, 2.05680735e-08, ...,
        3.46893003e-09, 1.37759990e-08, 2.55191435e-06],
       [5.29249007e-07, 6.24676177e-09, 1.48969370e-09, ...,
        1.39136418e-07, 4.45581236e-06, 2.96120197e-06],
       [1.40814240e-08, 6.71311284e-10, 5.23657550e-12, ...,
        1.33741503e-06, 2.74308178e-07, 4.38621464e-06],
       ...,
       [5.53973720e-08, 3.42950579e-09, 1.00011315e-08, ...,
        9.05878551e-06, 2.85068541e-06, 1.06095243e-07],
       [2.32562190e-12, 9.48925716e-10, 6.81325307e-09, ...,
        8.80663453e-09, 2.23348223e-07, 4.04246876e-08],
       [5.00154917e-10, 8.07001310e-09, 3.68304356e-08, ...,
        4.67868766e-09, 6.64208777e-09, 1.11203383e-06]], dtype=float32)

In [32]:
predict2 = []
for i in range(pred2.shape[0]):
    predict2.append(pred2[i].argmax())
predict2 = np.array(predict2)
predict2

array([47, 41, 41, ..., 22, 42, 47], dtype=int64)

In [33]:
print ('Here is the classification report:')
print (classification_report(y_test, predict2))

Here is the classification report:
              precision    recall  f1-score   support

           0       0.74      0.72      0.73       176
           1       0.89      0.64      0.74        25
           2       0.81      0.61      0.69        28
           3       0.00      0.00      0.00         1
           5       0.86      0.86      0.86        85
           7       0.90      0.92      0.91      2918
           8       0.85      0.58      0.69        69
           9       0.00      0.00      0.00         2
          10       0.85      0.81      0.83      1026
          11       0.93      0.86      0.89      1089
          12       0.77      0.82      0.79       299
          13       0.70      0.72      0.71       323
          14       0.82      0.77      0.79       378
          15       0.90      0.89      0.89       801
          16       0.45      0.28      0.34        18
          17       0.65      0.67      0.66       598
          18       0.44      0.20      0.28   

C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 모델개발용 데이터 예측 (digit_2)

In [34]:
logging.info('모델개발용 자료 digit_2 예측')

2022-04-13 01:36:26,663 - root - INFO - 모델개발용 자료 digit_2 예측


In [35]:
digit2_pred = model2.predict(test_data)
digit2_pred

array([[1.91625542e-08, 2.21354357e-09, 1.07710598e-07, ...,
        1.37286493e-08, 6.06073911e-08, 6.20841865e-06],
       [2.92120944e-07, 2.31803798e-09, 6.55163959e-08, ...,
        2.94064262e-06, 5.65277332e-07, 2.36343951e-07],
       [4.47696742e-08, 5.23249533e-09, 5.54991217e-08, ...,
        9.99602377e-01, 1.97504528e-06, 4.77241374e-05],
       ...,
       [1.14656764e-07, 1.30519351e-09, 2.37035236e-09, ...,
        1.96912268e-07, 7.17825333e-06, 4.45856531e-05],
       [9.31479320e-08, 4.44242615e-10, 3.96934396e-09, ...,
        7.17288367e-06, 9.46736023e-09, 1.52708653e-05],
       [2.51691472e-06, 4.68308379e-08, 9.12375867e-07, ...,
        5.29535115e-04, 4.68985081e-06, 1.23229896e-04]], dtype=float32)

In [36]:
digit2_predict = []
for i in range(digit2_pred.shape[0]):
    digit2_predict.append(digit2_pred[i].argmax())
digit2_predict = np.array(digit2_predict)
digit2_predict

array([47, 40, 71, ..., 41, 69, 57], dtype=int64)

In [37]:
digit2_predict = encoder2.inverse_transform(digit2_predict)
digit2_predict

array([56., 46., 94., ..., 47., 90., 68.])

# LSTM 학습 (digit_3)

In [38]:
logging.info('digit_3 학습')

2022-04-13 01:36:35,768 - root - INFO - digit_3 학습


In [39]:
target3 = train['digit_3']
# digit_3에 1개만 가지고 있는 카테고리 존재하므로 stratify=target3하면 에러 발생
X_train, X_test, y_train, y_test = train_test_split(train_data, target3, test_size=0.2, shuffle=True)

In [40]:
vocab_size = 10760
embedding_dim = 128
hidden_units = 128
num_classes = 225 # digit_3 225개 카테고리

model3 = Sequential()
model3.add(Embedding(vocab_size, embedding_dim))
model3.add(LSTM(hidden_units, return_sequences=True))
model3.add(LSTM(hidden_units))
model3.add(Dropout(0.2))
model3.add(Dense(hidden_units, activation='tanh'))
model3.add(Dense(num_classes, activation='softmax'))

In [41]:
# earlystopping은 위에 정의되었으므로 checkpoint만 정의
mc3 = ModelCheckpoint('best_model3.h5', monitor='val_accuracy', mode = 'auto', verbose=1, save_best_only=True)

In [42]:
model3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history3 = model3.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc3], validation_split=0.2)

Epoch 1/10
20000/20000 [==============================] - 249s 12ms/step - loss: 0.7203 - accuracy: 0.8233 - val_loss: 0.4169 - val_accuracy: 0.8937

Epoch 00001: val_accuracy improved from -inf to 0.89370, saving model to best_model3.h5
Epoch 2/10
20000/20000 [==============================] - 253s 13ms/step - loss: 0.3882 - accuracy: 0.8997 - val_loss: 0.3633 - val_accuracy: 0.9061

Epoch 00002: val_accuracy improved from 0.89370 to 0.90608, saving model to best_model3.h5
Epoch 3/10
20000/20000 [==============================] - 252s 13ms/step - loss: 0.3390 - accuracy: 0.9108 - val_loss: 0.3543 - val_accuracy: 0.9081

Epoch 00003: val_accuracy improved from 0.90608 to 0.90812, saving model to best_model3.h5
Epoch 4/10
20000/20000 [==============================] - 250s 13ms/step - loss: 0.3095 - accuracy: 0.9172 - val_loss: 0.3450 - val_accuracy: 0.9108

Epoch 00004: val_accuracy improved from 0.90812 to 0.91078, saving model to best_model3.h5
Epoch 5/10
20000/20000 [===============

In [43]:
print("테스트 정확도: %.4f" % (model3.evaluate(X_test, y_test)[1]))

6250/6250 [==============================] - 25s 4ms/step - loss: 0.3793 - accuracy: 0.9113
테스트 정확도: 0.9113


In [44]:
pred3 = model3.predict(X_test)
pred3

array([[2.8775269e-11, 1.9274854e-11, 3.5644287e-12, ..., 4.3376843e-12,
        1.9709182e-08, 1.3547105e-07],
       [8.8529492e-07, 6.7842706e-07, 2.5420679e-06, ..., 6.6587295e-06,
        2.4459110e-05, 8.0973469e-04],
       [5.9085209e-11, 3.5672328e-13, 1.4264279e-10, ..., 1.6722839e-15,
        2.9657141e-11, 4.7867808e-12],
       ...,
       [9.4860830e-08, 4.0570061e-08, 9.0867243e-09, ..., 1.0331454e-05,
        2.1838214e-07, 1.1351282e-06],
       [1.0293895e-09, 4.5614765e-08, 5.7656987e-11, ..., 1.3124403e-08,
        2.1571807e-08, 5.6025495e-09],
       [3.5986128e-07, 2.9418045e-08, 1.8597814e-07, ..., 3.1765248e-05,
        8.4453973e-07, 2.4934064e-07]], dtype=float32)

In [45]:
predict3 = []
for i in range(pred3.shape[0]):
    predict3.append(pred3[i].argmax())
predict3 = np.array(predict3)
predict3

array([147, 182, 201, ..., 120, 148,  81], dtype=int64)

In [46]:
print ('Here is the classification report:')
print (classification_report(y_test, predict3))

Here is the classification report:
              precision    recall  f1-score   support

           0       0.67      0.62      0.65        82
           1       0.87      0.75      0.81        53
           2       0.31      0.50      0.38        30
           3       0.59      0.70      0.64        27
           4       0.00      0.00      0.00         3
           5       0.73      0.48      0.58        23
           8       0.00      0.00      0.00         2
           9       0.70      0.60      0.65        35
          10       0.98      0.92      0.95        52
          12       0.69      0.80      0.75       133
          13       0.84      0.86      0.85       197
          14       0.67      0.60      0.63       218
          15       0.81      0.93      0.86       149
          16       0.50      0.16      0.24        19
          17       0.82      0.77      0.80       255
          18       0.88      0.88      0.88      1811
          19       0.83      0.70      0.76   

C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 모델개발용 데이터 예측 (digit_3)

In [47]:
logging.info('모델개발용 자료 digit_3 예측')

2022-04-13 02:19:24,809 - root - INFO - 모델개발용 자료 digit_3 예측


In [48]:
digit3_pred = model3.predict(test_data)
digit3_pred

array([[4.5779464e-11, 9.8811535e-11, 2.3729187e-12, ..., 5.7348262e-12,
        8.8199798e-08, 4.3648907e-08],
       [1.0991066e-08, 4.3618029e-11, 3.6106276e-09, ..., 5.3547950e-07,
        5.7488808e-10, 1.2982049e-08],
       [2.4743588e-10, 9.2898524e-09, 1.8151720e-10, ..., 3.3744507e-10,
        6.3231391e-06, 1.3851150e-06],
       ...,
       [2.5813802e-07, 2.0584719e-05, 4.4609926e-08, ..., 3.6329996e-06,
        1.7594053e-05, 6.4062748e-05],
       [3.6091736e-09, 7.0136483e-09, 7.0622415e-09, ..., 9.4707495e-09,
        1.2231033e-07, 1.6740410e-06],
       [3.7810317e-08, 6.6466242e-07, 1.7333912e-07, ..., 6.4274678e-07,
        4.2939300e-05, 3.5970946e-04]], dtype=float32)

In [49]:
digit3_predict = []
for i in range(digit3_pred.shape[0]):
    digit3_predict.append(digit3_pred[i].argmax())
digit3_predict = np.array(digit3_predict)
digit3_predict

array([147, 122, 219, ..., 132, 214, 169], dtype=int64)

In [50]:
digit3_predict = encoder3.inverse_transform(digit3_predict)
digit3_predict

array([561., 466., 949., ..., 478., 902., 682.])

# 답안 작성용 파일

In [51]:
submission = pd.read_csv('../답안 작성용 파일.csv', encoding='cp949')
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [52]:
submission['digit_1'] = digit1_predict
submission['digit_2'] = digit2_predict
submission['digit_3'] = digit3_predict

In [53]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56.0,561.0,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46.0,466.0,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94.0,949.0,절에서,신도을 대상으로,불교단체운영
3,id_000004,C,95.0,302.0,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56.0,562.0,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,A,47.0,869.0,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86.0,862.0,한의원에서,외래환자위주고,치료
99997,id_099998,G,47.0,478.0,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90.0,902.0,사업장에서,일반인.학생대상으로,학습공간제공


In [54]:
submission.to_csv('답안 작성용 파일(lstm).csv', index=False, encoding='utf-8-sig')